# Introduction

In [12]:
import ModelSearch
import pandas as pd 
import sys

sys.path.insert(0, '../sbmlcore')
sys.path.insert(0, '/Users/dylanadlard/Library/')
import sbmlcore from 


ModuleNotFoundError: No module named 'sbmlcore'

Hello[WHO2021]<a id='WHO2021'></a> Goodbye

[Blurf](#WHO2021)

# Methods and Results

### Data Processing Workflow

The pipeline below (FIGURE) first trains models on binary phenotype readouts (resistance vs susceptible) and leads to a prediction of a binary phenotype. This pipeline requires the pulldown of structural, biochemical, thermodynamic, and evolutionary features from RNAP and its mutations before merging these with filtered, prepated mutation data. These data and that for the filtered, prepared phonetype preadouts form the feature and target sets, respectively. 

The data is split into a 70:30 training and test set, the former of which undergoes a parameter grid search with cross validation and upsampling (discussed later). The model is retrained using the best parameters, and a classification report and confusion matrix is outputted post decision-threshold shifting to maximise specificity.

![title](images/direct_binary_class_workflow.png)

## Target Set Derivation

### Mutation data derivation

The data that formed the target and much of the training sets were derived from the CRyPTIC database. Mutation data was extracted from Illumina platform WGS of clinical isolates. Since this study focuses on predicting susceptiblity in response to solo structural SNPs in Mtb RNA polymerase, mutations associated with other drug targets, non-solo mutations, phylogenetic, and synonymous mutaations were filtered out, as well as insertion/deletion polymorphisms (indels), and mutations in the promoter. 

Of particular note is the fact mutations in the RRDR are reportedly responsible for more than 95% of resistance to rifampicin ([Mvelase2019](#Mvelase2019)); a statistic that is at least partially reflected in our data. Therefore, as this is a preliminary investigation, and for simplicity, only mutations located in rpoB are initially considered.

In [3]:
#read in CRyPTIC mutation data
mutation_df = pd.read_pickle('Data_tables/MUTATIONS-rnap.pkl.gz')
mutation_df.drop_duplicates(inplace=True)
#remove nulls and ensure mutation is a SNP

mutation_df = mutation_df[(~mutation_df['IS_NULL']) & (mutation_df['IS_FILTER_PASS']) & (mutation_df['IS_SNP'])]
#ensure mutation is in the coding region
mutation_df = mutation_df[mutation_df['IN_CDS']]
#ensure mutations are non-synonymous
mutation_df = mutation_df[~mutation_df['IS_SYNONYMOUS']]
#only investigate mutations in rpoB
mutation_df = mutation_df.loc[mutation_df.GENE=='rpoB']
#only investigate solo mutations
mutation_df = mutation_df.drop_duplicates(subset=['UNIQUEID'], keep=False)
#insert segid column
mutation_df['segid'] = [i[-1] for i in mutation_df.GENE]
mutation_df.set_index('UNIQUEID',verify_integrity=True, inplace=True)
mutation_df

,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,...,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,segid
UNIQUEID,,,,,,,,,,,,,,,,,,,,,
site.02.subj.1033.lab.2013185075.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,False,True,GENE,AAM,NaN,,,02,1,B
site.02.subj.0739.lab.2013221518.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,False,True,GENE,AAM,NaN,,,02,1,B
site.02.subj.0604.lab.241032-14.iso.1,rpoB,M587I,587.0,587.0,NaN,NaN,atg,ata,True,False,...,False,True,GENE,AAM,NaN,,,02,1,B
site.02.subj.0104.lab.22A057.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,False,True,GENE,AAM,NaN,,,02,1,B
site.02.subj.0885.lab.22A119.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,False,True,GENE,AAM,NaN,,,02,1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.10.subj.SATRL0117008_S16.lab.TRL0117008_S16.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,False,True,GENE,AAM,NaN,,,10,1,B
site.10.subj.SATRL0115669_S18.lab.TRL0115669_S18.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,False,True,GENE,AAM,NaN,,,10,1,B
site.10.subj.XD01227882.lab.XD01227882.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,False,True,GENE,AAM,NaN,,,10,1,B


### Phenotype data derivation

Primary phenotypic data were Minimum Inhibitory Concentrations (MIC) read from bespoke CRyPTIC UKMYC5 and UKMYC6 susceptibility plates on which the same clinical isolates that underwent WGS were plated. Data was pooled form laboraties in 27 countries across 65 continents, and to date, the WHO-endoresed CRYpTIC dataset contains MICs of 15,211 isolates to 13 different anti-TB drugs ([Brankin et al, 2021](#Brankin2021), [Walker et al, 2022](#Walker2022)).
   
Each UKMYC5/6 96-well microtitre plate contained 13 different 1st and 2nd line anti-TB compounds, including rifampicin, as well as two repurposed drugs (delamanid and bedaquiline) and two positive control wells. Plates were manufactured with the drugs freeze-dried onto each well following a doubling dilution series. After incubation for 14 days, MICs are read by a trained lab technician via either a Sensititre-Vizion Digital MIC viewing system or via a Mirrored Box [(Plate et al, 2018)](#Plate2018), and these are verified by an Automated Mycobacterial Growth Detection Algorithm (AMyGDA) ([Fowler et al, 2018](#Fowler2018); [Plate et al, 2018](#Plate2018)), as well as via a community science platform, BashTheBug. Phenotype readout disagreements between all three methods constituted low-quality phenotypes and were discarded for this study. 

The dataset contains a resistance vs susceptible binary phenotype calculated directly from MICs (using a critical concentration Epidemiological Cut-off (ECOFF) value of 1.0 mg/L for rifampicin) that acts as the target set for binary classification models ([CRyPTIC, 2019](#CRyPTIC2019)).

In [4]:
phenotype_df = pd.read_pickle('Data_tables/DST_MEASUREMENTS-rifamycins.pkl.gz')
#filter out rifamycin
phenotype_df = phenotype_df.loc[phenotype_df.DRUG=='RIF']
phenotype_df.drop_duplicates(subset=["UNIQUEID"], keep="first", inplace=True)
#filter out rifamycin
phenotype_df.set_index('UNIQUEID',verify_integrity=True, inplace=True)
phenotype_df

,DRUG,PHENOTYPE,SOURCE,METHOD_1,METHOD_2,METHOD_3,METHOD_CC,METHOD_MIC
UNIQUEID,,,,,,,,
site.06.subj.06TB_0290.lab.06MIL0881.iso.1,RIF,R,CRyPTIC,liquid media,microdilution plate,UKMYC5,0.5,>4
site.05.subj.PMFR-0724.lab.MFR-199.iso.1,RIF,S,CRyPTIC,liquid media,microdilution plate,UKMYC5,0.5,0.12
site.05.subj.PTAN-0168.lab.TAN-500.iso.1,RIF,S,CRyPTIC,liquid media,microdilution plate,UKMYC5,0.5,<=0.06
site.06.subj.MHL_0185-14.lab.06MIL0212.iso.1,RIF,R,CRyPTIC,liquid media,microdilution plate,UKMYC5,0.5,>4
site.06.subj.MHL_1323-15-R.lab.06MIL0060.iso.1,RIF,R,CRyPTIC,liquid media,microdilution plate,UKMYC5,0.5,>4
...,...,...,...,...,...,...,...,...
site.05.subj.PTAN-0340.lab.TAN-347.iso.1,RIF,S,CLIRES,MODS,None,None,None,None
site.05.subj.PTAN-0252.lab.TAN-578.iso.1,RIF,S,CLIRES,MODS,None,None,None,None
site.05.subj.PSLM-0841.lab.SLM-108.iso.1,RIF,R,CLIRES,solid media,None,None,None,None


### Merge phenotype and mutation dataframes

In [5]:
merged = mutation_df.join(phenotype_df[['PHENOTYPE']],how='inner')
merged = merged.loc[merged.GENE=='rpoB']
merged

,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,...,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,segid,PHENOTYPE
UNIQUEID,,,,,,,,,,,,,,,,,,,,,
site.02.subj.1033.lab.2013185075.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,True,GENE,AAM,NaN,,,02,1,B,R
site.02.subj.0739.lab.2013221518.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,True,GENE,AAM,NaN,,,02,1,B,R
site.02.subj.0604.lab.241032-14.iso.1,rpoB,M587I,587.0,587.0,NaN,NaN,atg,ata,True,False,...,True,GENE,AAM,NaN,,,02,1,B,S
site.02.subj.0104.lab.22A057.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,True,GENE,AAM,NaN,,,02,1,B,R
site.02.subj.0885.lab.22A119.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,True,GENE,AAM,NaN,,,02,1,B,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.10.subj.SATRL0116187_S19.lab.TRL0116187_S19.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,True,GENE,AAM,NaN,,,10,1,B,R
site.10.subj.SATRL0117008_S16.lab.TRL0117008_S16.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,True,GENE,AAM,NaN,,,10,1,B,R
site.10.subj.SATRL0115669_S18.lab.TRL0115669_S18.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,...,True,GENE,AAM,NaN,,,10,1,B,R


This dataframe now contains all isolates with the desired type of mutation in rpoB and corresponding binary resistance phenotype. However, the majority of these are isolates that have the same mutation, particulary S450L as demonstrated below. Rather than building an unbiased model, training on this data would result in heavily weighting the model on the few most commonly observed mutations.


In [8]:
pd.DataFrame(merged['MUTATION'].value_counts())[:10]

,MUTATION
S450L,6013
D435V,777
H445D,405
H445Y,341
L452P,150
L430P,143
S450W,138
H445L,138
D435Y,119
I491F,110


To account for this, all data was 'compressed' so that rather than training on isolates, the models were trained on unique mutations. A complication, however, arises from the fact many mutations do not consistently result in the same phenotype. To address this, an arbitrary threshold of 0.5 was set, whereby mutations that occur three or more times are classified as their majority phenotype, while mutations that occur at least once are classified as resistant or susceptible if all phenotypes are resistinat or suscpetible.

In [9]:
#create a crosstab df to calculate the number (and total) of phenotypes per mutation
ct = pd.crosstab([merged.MUTATION, merged.segid], merged.PHENOTYPE)
ct['total'] = ct['R'] + ct['S']
ct['prob_R'] = ct['R']/ct['total']
ct['prob_S'] = ct['S']/ct['total']

ct.sort_values(by='total',ascending=False)

def calculate_RS(row):
    #determine majority phenotype
    phenotype='U'
    if row['total'] >= 3:
        if row['prob_R'] > 0.5:
            phenotype='R'
        elif row['prob_S'] > 0.5:
            phenotype='S'
    elif row['total'] >=1:
        if row['R'] == row['total']:
            phenotype='R'
        elif row['S'] == row['total']:
            phenotype='S'

    return phenotype

#determine majority phenotype with calculate_RS()
ct['PHENOTYPE'] = ct.apply(calculate_RS, axis=1)
#filter for mutations that contain a majority phenotype
ct = ct[ct['PHENOTYPE'].isin(['R','S'])]

ct.reset_index(inplace=True, level = ['segid'])
ct

PHENOTYPE,segid,I,R,S,U,total,prob_R,prob_S,PHENOTYPE
MUTATION,,,,,,,,,
A1002P,B,0,0,3,0,3,0.0,1.0,S
A286V,B,0,2,0,0,2,1.0,0.0,R
A29T,B,0,0,1,0,1,0.0,1.0,S
A29V,B,0,0,1,0,1,0.0,1.0,S
A334D,B,0,0,1,0,1,0.0,1.0,S
...,...,...,...,...,...,...,...,...,...
V895F,B,0,0,1,0,1,0.0,1.0,S
V946G,B,0,0,1,0,1,0.0,1.0,S
Y308C,B,0,1,0,0,1,1.0,0.0,R


In [10]:
#create the final dataframe, phenotypes for which have been determined by crosstab

mutation = [i for i in ct.index]
index = [i for i in range(0, len(ct))]
segid = [i for i in ct.segid]
phenotype = [i for i in ct.PHENOTYPE]
df = pd.DataFrame({'MUTATION':mutation, 'SEGID':segid, 'PHENOTYPE':phenotype}, index=index)
df

,MUTATION,SEGID,PHENOTYPE
0,A1002P,B,S
1,A286V,B,R
2,A29T,B,S
3,A29V,B,S
4,A334D,B,S
...,...,...,...
312,V895F,B,S
313,V946G,B,S
314,Y308C,B,R
315,Y572C,B,S


## Feature Set Derivation

The crystal structure of Mtb RNAP used in this study, as well as the structure from which trajectories were previously built, was recently solved to 3.8Å ([Lin et al, 2017](#Lin2017)). Relevant structural features include a bound DNA-mimicking nucleiv acid scaffold, a synthetic RNA oligomer corresponding to a 2-nt mRNA product, a bound rifampicin molecule, and σA responsible for binding the holoenzyme and modulating transcription initiation.

All in-silico structural feature pulldowns were performed using a suite of classes, termed sbmlcore (https://github.com/fowler-lab/sbmlcore); the development of which constituted a large component of the project, and the application of which is demonstrated below. Each class is first instantiated with desired input parameters and the output can be appended to a feature set using a public method within the class. Therefore, a user can add a new strucutral feature to the feature set within 2 lines of code, thereby allowing for a streamlined pipeline with rapid construction of a dataframe ready to be split for machine learning.

#### Biochemical Differences

Selected mutations-specific biochemical differences include changes in volume, molecular weight, hydropahty and isoelectric point

# Bibliography

Brankin A, Malone KM, Barilar I, Battaglia S, Pires Brandao A, Maurizio Cabibbe A, Carter J, Maria D, Claxton P, Clifton DA, et al (2021) A data compendium of Mycobacterium tuberculosis 1 antibiotic resistance.<a id='Brankin2021'></a>    

CRyPTIC (2021) A generalisable approach to drug susceptibility prediction for M. tuberculosis using machine learning and whole-genome sequencing.<a id='CRyPTIC2021'></a>     

CRyPTIC (2019) Epidemiological cutoffs for a 96-well broth microtitre plate for high-throughput research antibiotic susceptibility testing of M.tuberculosis.<a id='CRyPTIC2019'></a>

Fowler PW, Cruz ALG, Hoosdally SJ, Jarrett L, Borroni E, Chiacchiaretta M, Rathod P, Lehmann S, Molodtsov N, Walker TM, et al (2018) Automated detection of bacterial growth on 96-well plates for high-throughput drug susceptibility testing of mycobacterium tuberculosis. Microbiology (United Kingdom) 164: 1522–1530<a id=Fowler2018></a>

Lin W, Mandal S, Degen D, Liu Y, Ebright YW, Li S, Feng Y, Zhang Y, Mandal S, Jiang Y, et al (2017) Structural Basis of Mycobacterium tuberculosis Transcription and Transcription Inhibition. Molecular Cell 66: 169-179.e8<a id=Lin2017></a>

Mvelase NR, Pillay M, Sibanda W, Ngozo JN, Brust JCM & Mlisana KP (2019) RpoB Mutations Causing Discordant Rifampicin Susceptibility in Mycobacterium tuberculosis: Retrospective analysis of prevalence, phenotypic, genotypic, and treatment outcomes. Open Forum Infectious Diseases 6 <a id='Mvelase2019'></a>

Plate M, Bedaquiline C, Walker TM, Grazian C, Davies TJ, Peto TEA, Crook DW, Fowler PW & Cirillo DM (2018) Validating a 14-Drug Microtiter Plate Containing Bedaquiline and Delamanid for Large-Scale Research Susceptibility Testing of Mycobacterium tuberculosis. 62: 1–15<a id='Plate2018'></a>      

Walker TM, Fowler PW, Knaggs J, Hunt M, Peto TE, Walker AS, Crook DW, Walker TM, Miotto P, Cirillo DM, et al (2022) The 2021 WHO catalogue of Mycobacterium tuberculosis complex mutations associated with drug resistance: a genotypic analysis. The Lancet Microbe 3: e265–e273<a id='Walker2022'></a>   


[CRyPTIC, 2021](#CRyPTIC,2021)